The goal of this notebook is to explore the "delta kick model", which is a way of working with the IDE where we decompose a laser pulse into a series of delta-function kicks. This lets us look at how time delays between parts of the pulse directly couple to the kernel, causing intereference and therefore changing the resulting ionization probability.

Most of the delta kick code was actually moved into the library a long time ago, so we'll just access it from there.

In [1]:
import logging
import os
import functools
import datetime

import numpy as np

import simulacra as si
import simulacra.units as u

import ionization as ion
import ionization.ide.delta_kicks as dk

LOGMAN = si.utils.LogManager('simulacra', 'ionization', stdout_level = logging.INFO)
THIS_DIR = os.getcwd()
OUT_DIR = os.path.join(THIS_DIR, 'out', 'DeltaKickModel')
SIM_LIB = os.path.join(OUT_DIR, 'SIMLIB')

PLOT_KWARGS = dict(
    show = True,
    save = True,
    target_dir = OUT_DIR,
    fig_dpi_scale = 1.5,
)

CHECKPOINT_KWARGS = dict(
    checkpoints = True,
    checkpoint_dir = SIM_LIB,
    checkpoint_every = datetime.timedelta(minutes = 1),
)

%matplotlib inline

The first thing we'll look at is a basic "sine-like" delta-kick. It's made of two kicks of opposite polarity. We'll center them around $t=0$ for convenience.

In [2]:
sine_kicks = dk.DeltaKicks([
    dk.delta_kick(-100 * u.asec, -.1 * u.atomic_electric_field * u.atomic_time),
    dk.delta_kick(1100 * u.asec, .1 * u.atomic_electric_field * u.atomic_time),
])

sine_kick_sim = dk.DeltaKickSpecification(
    'sine_kick',
    time_initial = -200 * u.asec,
    time_final = 200 *  u.asec,
    electric_potential = sine_kicks,
).to_sim()

sine_kick_sim.run()
print(sine_kick_sim.info())

DeltaKickSimulation(sine_kick) {finished}
├─ UUID: e0b5fb17-fb95-4254-b198-c7b26e1298b9
├─ Status: finished
│  ├─ Initialization Time: 2018-03-02 03:36:02.803930
│  ├─ Latest Run Time: 2018-03-02 03:36:02.804431
│  ├─ End Time: 2018-03-02 03:36:04.277261
│  ├─ Running Time: 0:00:01.472830
│  └─ Elapsed Time: 0:00:01.473331
├─ DeltaKickSpecification(sine_kick)
│  ├─ UUID: 35fc600f-ce2c-458b-8d97-2bd4a33464d3
│  ├─ Time Evolution
│  │  ├─ Initial Time: -200.0 as
│  │  └─ Final Time: 200.0 as
│  ├─ Evolution Algorithm
│  │  └─ Evolution Gauge: LEN
│  └─ IDE Parameters
│     ├─ Initial State: a = 1
│     ├─ Prefactor: -2.3081736167869146e+30
│     ├─ LengthGaugeHydrogenKernel
│     │  └─ Bound State Energy: -13.606 eV | -0.5 hartree
│     ├─ DC Correct Electric Field: yes
│     └─ Electric Potential: DeltaKicks
│        ├─ Number of Kicks: 2
│        └─ Maximum Kick Amplitude: 1.2438403206875851e-06
└─ DeltaKickSpecification(sine_kick)
   ├─ UUID: 35fc600f-ce2c-458b-8d97-2bd4a33464d3
   ├─

In [5]:
print(sine_kick_sim.b2)

[1.         0.98029605 0.96077745 0.96077745]
